In [28]:
import cv2
import os
import pandas as pd
from PIL import Image
from IPython.display import display
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, LabelBinarizer, LabelEncoder
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Dropout, Flatten, Dense
import numpy as np

In [49]:
# Crear dataframes de train y test através de las carpetas de fotos

dataset = []

for label in os.listdir('dataset'):
    print(label)
    if not os.path.isdir(f'dataset/{label}'):
        continue
    for image_filename in os.listdir(f'dataset/{label}'):
        image = Image.open(f'dataset/{label}/{image_filename}').convert("RGB")
        pixels = np.array(image).astype(float)
        dataset.append((pixels, label))
            
# Convertir la llista de dataset en dataframe

df = pd.DataFrame(dataset, columns=['pixels', 'label'])

# hacer otro archivo con esta funcion


happy
.DS_Store
sad
fear
surprise
neutral
angry
disgust


In [50]:
df.to_pickle('images.pkl')

In [51]:
df = pd.read_pickle('images.pkl')

In [34]:
df.head()

,pixels,label
0,"[[[108.0, 108.0, 108.0], [83.0, 83.0, 83.0], [...",happy
1,"[[[137.0, 137.0, 137.0], [142.0, 142.0, 142.0]...",happy
2,"[[[111.0, 111.0, 111.0], [148.0, 148.0, 148.0]...",happy
3,"[[[151.0, 151.0, 151.0], [156.0, 156.0, 156.0]...",happy
4,"[[[5.0, 5.0, 5.0], [4.0, 4.0, 4.0], [5.0, 5.0,...",happy


In [52]:
df['label'].unique()

array(['happy', 'sad', 'fear', 'surprise', 'neutral', 'angry', 'disgust'],
      dtype=object)

In [53]:
# Asigne 'X' and 'y' and Normalize data

X = np.array((df['pixels']/255).values.tolist())
y = np.array(LabelBinarizer().fit_transform(df['label']))


In [54]:
# Dividir en train y test 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [43]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(28709, 128, 128, 3)
(28709, 7)
(7178, 128, 128, 3)
(7178, 7)


In [21]:
# Definir Modelo
IMAGE_HEIGHT = 48
IMAGE_WIDTH = 48
CHANNELS_LIST = 1
NUM_LABELS = 7

model = Sequential([
    Conv2D(32,(3,3), input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, CHANNELS_LIST), activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dense(NUM_LABELS, activation='softmax')
])


# Compilar los datos
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

2022-12-26 17:06:58.556106: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [22]:
# Train
model.fit(X_train, y_train, batch_size=32, epochs=10)

Epoch 1/10
898/898 [==============================] - 159s 175ms/step - loss: 1.7234 - accuracy: 0.3537
Epoch 2/10
898/898 [==============================] - 149s 166ms/step - loss: 1.3778 - accuracy: 0.4729
Epoch 3/10
898/898 [==============================] - 152s 170ms/step - loss: 1.1995 - accuracy: 0.5461
Epoch 4/10
898/898 [==============================] - 131s 146ms/step - loss: 1.0050 - accuracy: 0.6305
Epoch 5/10
898/898 [==============================] - 134s 150ms/step - loss: 0.7805 - accuracy: 0.7202
Epoch 6/10
898/898 [==============================] - 132s 147ms/step - loss: 0.5448 - accuracy: 0.8155
Epoch 7/10
898/898 [==============================] - 139s 155ms/step - loss: 0.3399 - accuracy: 0.8938
Epoch 8/10
898/898 [==============================] - 143s 159ms/step - loss: 0.1943 - accuracy: 0.9446
Epoch 9/10
898/898 [==============================] - 150s 167ms/step - loss: 0.1098 - accuracy: 0.9735
Epoch 10/10
898/898 [==============================] - 167s 186m

In [23]:
score, acc = model.evaluate(X_test, y_test, batch_size = 100)
print('Test Score: ', score)
print('Test Acucuracy: ', acc)

72/72 [==============================] - 4s 53ms/step - loss: 3.2950 - accuracy: 0.4451
Test Score:  3.2950046062469482
Test Acucuracy:  0.4451100528240204


In [45]:

#modelo MobileNet
base_model = tf.keras.applications.MobileNetV2(
        input_shape=(48, 48, 3),
        weights='imagenet',  # Utilizamos los pesos preintrenados en ImageNet
        include_top=False  # No incluimos la capa densa final
    )
base_model.trainable = False

output = base_model.output
output = tf.keras.layers.BatchNormalization()(output)
output = tf.keras.layers.Dropout(0.5)(output)
output = tf.keras.layers.Dense(1, activation='sigmoid')(output)

model = tf.keras.models.Model(base_model.input, output)


model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

9406464/9406464 [==============================] - 1s 0us/step


In [46]:
from tensorflow.keras.callbacks import TensorBoard
tensorboarddata = TensorBoard(log_dir='logs/train_borad')

In [47]:
%load_ext tensorboard
%tensorboard --logdir logs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 5010), started 0:01:38 ago. (Use '!kill 5010' to kill it.)

In [48]:
model.fit(X_train, y_train, 
          batch_size=32, 
          epochs=50,
          callbacks=[tensorboarddata])

Epoch 1/50


ValueError: in user code:

    File "/usr/local/Caskroom/miniconda/base/envs/core/lib/python3.10/site-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/Caskroom/miniconda/base/envs/core/lib/python3.10/site-packages/keras/engine/training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/Caskroom/miniconda/base/envs/core/lib/python3.10/site-packages/keras/engine/training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/Caskroom/miniconda/base/envs/core/lib/python3.10/site-packages/keras/engine/training.py", line 1023, in train_step
        y_pred = self(x, training=True)
    File "/usr/local/Caskroom/miniconda/base/envs/core/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/usr/loc